# Load the "Add Utility Script" from File Menu and search for my "Load Kaggle" script

In [ ]:
from load_kaggle import load_kaggle

In [ ]:
subm, train, test = load_kaggle()
print(train.shape, test.shape, subm.shape)
train.head()

In [ ]:
targets = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
len(targets)

In [ ]:
# To perform feature engg, we need Featurewiz, an autoML library for feature engg
!pip install featurewiz

# we need to create time series features from date-time column

In [ ]:
import featurewiz as FW

In [ ]:
ts_column = 'date_time'
train1, ts_cols1 = FW.FE_create_time_series_features(train, ts_column)

In [ ]:
### we need to make sure the same columns are created in test ###
test1, ts_cols2 = FW.FE_create_time_series_features(test, ts_column, ts_cols1)

In [ ]:
### we now need to make sure that train and test have the same columns ##

In [ ]:
rem_cols = [x for x in list(train) if x not in [ts_column]+targets]
rem_cols

In [ ]:
train2 = train1[rem_cols+ts_cols2+targets]
train2.shape

In [ ]:
test2 = test1[rem_cols+ts_cols2]
test2.shape

In [ ]:
train3, test3 = FW.FE_convert_all_object_columns_to_numeric(train2, test2)

In [ ]:
train3.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
#BL = RandomForestRegressor(n_estimators=100)
BL = CatBoostRegressor(objective='MultiRMSE')
BL

In [ ]:
BL.fit(train3[rem_cols+ts_cols2], train3[targets])

In [ ]:
y_preds = BL.predict(test3[rem_cols+ts_cols2])
y_preds[:4]

In [ ]:
subm[targets] = y_preds
subm.head()

In [ ]:
subm.to_csv('submission_Cat_FW.csv', index=False)